In [ ]:
import os
import sys
import subprocess

# Clone the repository
!rm -rf A-scab/
!git clone -b hackathon https://github.com/WUR-AI/A-scab.git

# Change directory
%cd A-scab

# Get the absolute path of the project root *after* changing directory
PROJECT_ROOT = os.getcwd()

# Install poetry if needed.
!pip install -qqq poetry

# Configure poetry to create virtual environments in the project directory
!poetry config virtualenvs.in-project true

# Install project dependencies
# This step may take some time (e.g., 5 minutes)
!poetry install --quiet --all-extras

# These are things you don't need to know about for now :)

# 1. Add the project root to sys.path
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
    print(f"Added project root {PROJECT_ROOT} to sys.path")

# 2. Get the path to the site-packages directory of the poetry virtual environment
try:
    venv_path_output = subprocess.check_output(['poetry', 'env', 'info', '--path']).decode('utf-8').strip()
    python_version_major_minor = f"python{sys.version_info.major}.{sys.version_info.minor}"
    SITE_PACKAGES_PATH = os.path.join(venv_path_output, 'lib', python_version_major_minor, 'site-packages')
except Exception as e:
    print(f"Could not determine poetry venv site-packages path using 'poetry env info': {e}")
    print("Attempting a common fallback path structure...")
    SITE_PACKAGES_PATH = os.path.abspath(os.path.join('.venv', 'lib', f'python{sys.version_info.major}.{sys.version_info.minor}', 'site-packages'))

# 3. Add the site-packages directory to sys.path
if os.path.exists(SITE_PACKAGES_PATH) and SITE_PACKAGES_PATH not in sys.path:
    sys.path.insert(0, SITE_PACKAGES_PATH)
    print(f"Added {SITE_PACKAGES_PATH} to sys.path")
else:
    print(f"Warning: Could not find site-packages at {SITE_PACKAGES_PATH} or it's already in sys.path.")

import ascab
import gymnasium as gym

In [ ]:
# RL agent

# First upload your agent, then run it

from ascab.train import RLAgent
from ascab.utils.plot import plot_results
from stable_baselines3 import PPO, SAC, TD3, DQN, HER

ascab_val = gym.make('AscabValEnv-Discrete')
path_model='/content/rl_agent'
algo=DQN
observation_filter=list(ascab_val.observation_space.keys())

rl_agent = RLAgent(
    ascab_test=ascab_val,
    observation_filter=list(ascab_val.observation_space.keys()),
    path_model=path_model,
    rl_algorithm=algo
)
results = rl_agent.run()

In [10]:
# Conditional agent

from datetime import datetime
from ascab.train import BaseAgent
from ascab.env.env import AScabEnv

ascab_val = gym.make('AscabValEnv-Continuous-NonRL')

# First define the class, not forgetting to SubClass `BaseAgent`
# In general, you want to change the `get_action` method to apply your conditional spraying strategy
class HowMyLocalFarmerSprays(BaseAgent):
    def __init__(
        self,
        ascab: AScabEnv = None,
        render: bool = False,
    ):
        super().__init__(ascab=ascab, render=render)

    def get_action(self, observation: dict = None) -> float:
        # The code below means: "If it is forecasted that it will rain in two days, I will spray today".
        if self.ascab.get_wrapper_attr("info")["Forecast_day2_HasRain"] and self.ascab.get_wrapper_attr("info")["Forecast_day2_HasRain"][-1]:
            return 1.0  # the agent sprays this much if is forecasted to rain in two days
        return 0.0

farmer_strategy = HowMyLocalFarmerSprays(ascab_val)

# Use the class method below to run your agent!
results = farmer_strategy.run()

Reward: -0.3506618066521444
Reward: -0.45003198230700403
Reward: -0.4000660617666953
Reward: -0.2750000104282679


In [ ]:
dict_to_plot = {"Results":results}
plot_results(dict_to_plot, save_path=os.getcwd())